##<8-3 프리트레인 마친 모델로 문장 생성하기 >   


이번 실습에서는 프리트레인을 마친 GPT 모델을 가지고 문장을 생성해 보겠습니다. 8-1절에서 이미 살펴봤던 것처럼 GPT 모델의 프리트레인 태스크는 '다음 단어 맞히기'이므로 파인튜닝을 수행하지 않고도 프리트레인을 마친 GPT 모델만으로 문장을 생성해 볼 수가 있습니다.   
실습 대상 모델은 KoGPT2입니다.


###GPT 모델로 문장 생성하기   

코랩에 접속한 후 `[내 드라이브에 복사]`를 진행하고 [런타임 $\rightarrow$ `런타임 유형 변경`] 메뉴에서 하드웨어 가속을 사용하지 않도록 `[None]`을 선택합니다.

<**1단계**> **모델 초기화하기**    

In [7]:
!pip install ratsnlp

그리고 다음 코드를 실행해 프리트레인을 마친 KoGPT2 모델(`skt/kogpt2-base-v2`)을 읽어들입니다. `model.eval()` 함수를 실행하면 드롭아웃 등 학습 때만 필요한 기능들을 꺼서 평가모드로 동작하도록 해줍니다.

In [8]:
#체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


이어서 KoGPT2의 토크나이저를 선언합니다.

In [9]:
#토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


다음 코드를 실행해서 KoGPT2 모델에 넣을 입력값, 즉 컨텍스트(프롬프트)를 만들 수 있습니다. 토크나이저의 `encode()` 메서드는 입력 문장을 토큰화한 뒤 정수로 인덱싱하는 역할을 수행합니다. `return_tensors` 인자를 `pt`로 주면 인덱싱 결과를 파이토치의 텐서 자료형으로 반환합니다.

In [10]:
#모델 입력값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [11]:
input_ids

tensor([[25906,  8702,  7801,  8084]])

`안녕하세요`라는 문자열이 4개의 정수로 구성된 파이토치 텐서로 변환된 결과를 확인할 수 있습니다.

이제는 입력값(`input_ids`)을 `안녕하세요`로 통일해 보겠습니다. 다시 말하면 `안녕하세요`를 모델에 입력해 이후 문장을 생성해보는 것입니다.

<**2단계**> **그리디 서치하기**   

2장에서 이미 살펴봤듯이 언어 모델은 컨텍스트(토큰 시퀀스)를 입력받아 다음 토큰이 나타날 확률을 출력으로 반환합니다. 모델의 출력 확률 분포로부터 다음 토큰을 반복해서 선택하는 과정이 바로 문장 생성 태스크가 됩니다.     

하지만 문제는 특정 컨텍스트 다음에 올 토큰으로 무수히 많은 경우의 수가 존재한다는 것입니다. 다음 토큰이 어떤 것이 되느냐에 따라서 생성되는 문장의 의미가 180도 달라질 수 있는 것이죠.  
예를 들어 다음 그림은 `그`라는 컨텍스트에서 출발해 다음 토큰으로 어떤 것이 적절한지 언어 모델이 예측한 결과이며, 오른쪽은 그로부터 추려 낸 문장 후보를 나타낸 것입니다. 띄어쓰기는 이해하기 쉽도록 임의로 넣었습니다. 모두 9가지입니다.

<center><그림 - '그' 다음 토큰 예측 결과></center>

<p align="center"><img src="https://i.imgur.com/JhcrUp8.jpg">


문장을 생성할 때 언어 모델의 입력값은 컨텍스트이고 출력값은 컨텍스트 다음에 오는 단어의 확률 분포라는 점을 떠올려 봅시다. `그`로부터 출발해 네 번째 토큰으로 적절한 것이 무엇일지 선택하려면 앞의 9가지 모든 케이스를 모델에 입력해서 다음 토큰 확률 분포를 계산해보아야 합니다.

이해가 쉽도록 모델의 예측 결과를 단순화해서 적었지만 실제로는 9가지보다 훨씬 많은 케이스가 존재할 것입니다. 이론적으로는 다음 단어를 하나 선택해야 할 때 어휘 집합 크기만큼의 경우의 수가 생길 수 있습니다. 이렇게 반복적으로 다음 토큰을 생성할 경우 무수히 많은 가짓수가 파생되며 모든 경우의 수를 계산해 보는 것은 사실상 불가능합니다.

***그리드 서치(greedy search)*** 는 이러한 문제의 대안으로 제시되었습니다. 매 순간 최선을 선택해 탐색 범위를 줄여 보자는 것이 핵심입니다. 그리디 서치를 앞의 예시에 적용해 보면 모델은 다음처럼 `그 책이`를 생성 결과로 내놓게 됩니다.

<center><그림 - 그리디 서치></center>

<p align="center"><img src="https://i.imgur.com/YwqI1bg.jpg">   

그림 출처 : ratsgo

첫 번째 단어 예측 결과에서 `책`이 0.5로 가장 높으므로 다음 단어로 `책`을 선택하고 모델에 입력해 다음 단어 확률 분포를 계산합니다. 마찬가지로 `이`, `을`, `읽` 셋 가운데 확률값이 가장 높은 `이`(0.4)를 그다음 단어로 선택합니다.

이 그리디 서치를 진행 중인 실습에 적용해 보겠습니다. 그리디 서치를 수행하는 다음 코드에서 핵심 인자는 `do_sample=False`입니다. `max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 `EOS(end of sentence)` 등 스페셜 토큰이 나타나면 생성을 멈춥니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 스페셜 토큰이 등장해 생성이 멈추면 해당 토큰이 나올 확률을 0으로 수정하여 적어도 해당 길이를 만족할 때까지는 문장 생성이 종료되지 않도록 합니다.

In [12]:
#그리드 서치
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )